In [70]:
import numpy as np
import pandas as pd
import glob
from udfs import ipca

# Ajustando dados e trazendo tudo a valor presente

In [71]:
files = glob.glob("anac.nosync/*.CSV")

In [72]:
# %%time
# for file in files:
#     df = pd.read_csv(file, sep=";", encoding='latin1')
#     df = df.loc[:, ~df.columns.str.contains('^Unnamed')]
#     df.columns = ["ANO","MES","EMPRESA","ORIGEM","DESTINO","TARIFA","ASSENTOS"]
#     df.TARIFA = df.TARIFA.str.replace(",", ".").astype(float).copy()
#     df['REAL'] = df.apply(lambda row: ipca(row["TARIFA"], f"{row['MES']}/{row['ANO']}", "12/2022")[0], axis=1)
#     df.to_csv(f"anac.nosync/real/{file.split("/")[1]}.csv", index=False, sep=";")

# Começando com o Spark

In [73]:
from pyspark.sql import SparkSession
from pyspark.conf import SparkConf
import pyspark.sql.functions as f
from pyspark.sql.types import *


conf = SparkConf()
# conf.set("spark.eventLog.enabled", False)
# conf.set("spark.eventLog.dir", "/opt/spark/spark-events")
# conf.set("spark.history.fs.logDirectory", "/opt/spark/spark-events")
# conf.set("spark.executor.memory", "1024m")
# conf.set("spark.sql.autoBroadcastJoinThreshold", -1)
# conf.set("spark.sparkContext.setLogLevel", "ERROR")

spark = SparkSession.\
        builder.\
        appName("pyspark-anac").\
        master("spark://spark-master:7077").\
        config(conf=conf).\
        getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/04/11 20:49:32 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [74]:
spark
# sc = spark.sparkContext
# sc.addPyFile("/opt/notebooks/udfs.py")

In [75]:
schema = StructType(
    [
        StructField('ANO', IntegerType(), True), 
        StructField('MES', IntegerType(), True), 
        StructField('EMPRESA', StringType(), True), 
        StructField('ORIGEM', StringType(), True), 
        StructField('DESTINO', StringType(), True), 
        StructField('TARIFA', FloatType(), True), 
        StructField('ASSENTOS', IntegerType(), True),
        StructField('REAL', FloatType(), True),
    ]
)

In [76]:
sdf = spark.read.csv('anac.nosync/real/*.csv', header=True, sep=";", schema=schema)

In [77]:
sdf.schema

StructType([StructField('ANO', IntegerType(), True), StructField('MES', IntegerType(), True), StructField('EMPRESA', StringType(), True), StructField('ORIGEM', StringType(), True), StructField('DESTINO', StringType(), True), StructField('TARIFA', FloatType(), True), StructField('ASSENTOS', IntegerType(), True), StructField('REAL', FloatType(), True)])

In [78]:
sdf.count()

55291652

In [79]:
sdf.limit(3).show()

+----+---+-------+------+-------+------+--------+------+
| ANO|MES|EMPRESA|ORIGEM|DESTINO|TARIFA|ASSENTOS|  REAL|
+----+---+-------+------+-------+------+--------+------+
|2021| 10|    AZU|  SBAC|   SBAR| 565.9|       1|616.34|
|2021| 10|    AZU|  SBAC|   SBBR| 409.9|       1|446.44|
|2021| 10|    AZU|  SBAC|   SBBV| 748.9|       4|815.66|
+----+---+-------+------+-------+------+--------+------+



# Ajustar todos os valanac.nosync/ pela inflação de Dezembro de 2022

Nao funciona aqui por questão de processamento.


def valor_real(value, date_start):
    return ipca(value, date_start, "12/2022")[0]

vreal = f.udf(valor_real, DoubleType())    # registering as udf here


sdf = sdf.withColumn(
    'REAL',
    vreal(
        f.col("TARIFA"), 
        f.concat(f.col("MES"), f.lit("/"), f.col("ANO")),
    )
)

sdf = sdf.select("ANO", "MES", "ASSENTOS", "REAL")


# Dado final

In [80]:
data = sdf.groupby("ANO", "MES").agg(
    f.sum("ASSENTOS").alias("TOTAL"),
    f.collect_list(f.col('REAL')),
    f.collect_list(f.col('ASSENTOS')),
)

In [81]:
def get_average(values, weights):
    return float(np.average(values, weights=weights))    # function created to calculate wt. median

waverage = f.udf(get_average, DoubleType())    # registering as udf here

data = data.withColumn(
    'Weighted_average',
    waverage(
        f.col("collect_list(REAL)"), f.col("collect_list(ASSENTOS)")
    )
)

In [82]:
data.cache()

DataFrame[ANO: int, MES: int, TOTAL: bigint, collect_list(REAL): array<float>, collect_list(ASSENTOS): array<int>, Weighted_average: double]

In [83]:
data.limit(3).show()

+----+---+-------+--------------------+----------------------+-----------------+
| ANO|MES|  TOTAL|  collect_list(REAL)|collect_list(ASSENTOS)| Weighted_average|
+----+---+-------+--------------------+----------------------+-----------------+
|2022| 10|2450901|[1067.1, 660.59, ...|  [1, 2, 3, 26, 8, ...|648.7533683010134|
|2012| 10|4443441|[553.07, 1109.43,...|  [2, 1, 1, 2, 1, 7...|567.5236231638822|
|2007|  6|1366528|[459.67, 1140.66,...|  [103, 10, 56, 7, ...|681.8399205044295|
+----+---+-------+--------------------+----------------------+-----------------+



In [84]:
data.select("ANO", "MES", "TOTAL", "Weighted_average")\
    .repartition(1)\
    .write.format("com.databricks.spark.csv")\
    .option("header", "true")\
    .save("tarifa_real.csv")

AnalysisException: path file:/opt/notebooks/anac/tarifa_real.csv already exists.

In [ ]:
df_final = data.select("ANO", "MES", "TOTAL", "Weighted_average").toPandas()

In [ ]:
df.sort_values(["ANO", "MES"], ascending=False)

# Fechando aqui

In [1]:
import pandas as pd
import numpy as np

In [2]:
from monetary_correction import ipca

In [4]:
corretor = ipca

# Outubro

In [20]:
outubro = pd.read_csv("anac.nosync/202210.CSV", sep=";", encoding="latin1")
outubro.columns = ['ANO', 'MES', 'EMPRESA', 'ORIGEM', 'DESTINO', 'TARIFA', 'ASSENTOS']
outubro.TARIFA = outubro.TARIFA.str.replace(",", ".").astype(float)

In [48]:
# Aqui com os dados base julho de 2022
wm = lambda x: np.average(x, weights=outubro.loc[x.index, "ASSENTOS"])

media_outubro = outubro.groupby(['ANO', 'MES']).agg(price_weighted_mean=("TARIFA", wm))
media_outubro

,,price_weighted_mean
ANO,MES,
2022,10,638.356865


In [41]:
outubro_2019 = pd.read_csv("anac.nosync/201910.CSV", sep=";", encoding="latin1")
outubro_2019.TARIFA = outubro_2019.TARIFA.str.replace(",", ".").astype(float)
outubro_2019['REAL'] = outubro_2019.apply(lambda row: corretor(row["TARIFA"], f"{int(row.MES)}/{int(row.ANO)}", "10/2022")[0], axis=1) 

In [44]:
# Aqui com os dados base julho de 2022
wm = lambda x: np.average(x, weights=outubro_2019.loc[x.index, "ASSENTOS"])

media_outubro_2019 = outubro_2019.groupby(['ANO', 'MES']).agg(price_weighted_mean=("REAL", wm))
media_outubro_2019

,,price_weighted_mean
ANO,MES,
2019,10,560.817476


In [51]:
(float(media_outubro['price_weighted_mean']) - float(media_outubro_2019['price_weighted_mean'])) / float(media_outubro_2019['price_weighted_mean']) *100

13.82613636966195

# Dezembro 22

In [58]:
dezembro = pd.read_csv("anac.nosync/202212.CSV", sep=";", encoding="latin1")
# dezembro.columns = ['ANO', 'MES', 'EMPRESA', 'ORIGEM', 'DESTINO', 'TARIFA', 'ASSENTOS']
dezembro.TARIFA = dezembro.TARIFA.str.replace(",", ".").astype(float)


In [67]:
# Aqui com os dados base julho de 2022
wm = lambda x: np.average(x, weights=dezembro.loc[x.index, "ASSENTOS"])

media_dezembro = dezembro.groupby(['ANO', 'MES']).agg(price_weighted_mean=("TARIFA", wm))
media_dezembro

,,price_weighted_mean
ANO,MES,
2022,12,650.817921


In [63]:
dezembro_2019 = pd.read_csv("anac.nosync/201912.CSV", sep=";", encoding="latin1")
dezembro_2019.columns = ['ANO', 'MES', 'EMPRESA', 'ORIGEM', 'DESTINO', 'TARIFA', 'ASSENTOS']
dezembro_2019.TARIFA = dezembro_2019.TARIFA.str.replace(",", ".").astype(float)

In [65]:
dezembro_2019['REAL'] = dezembro_2019.apply(lambda row: corretor(row["TARIFA"], f"{int(row.MES)}/{int(row.ANO)}", "10/2022")[0], axis=1) 

In [66]:
# Aqui com os dados base julho de 2022
wm = lambda x: np.average(x, weights=dezembro_2019.loc[x.index, "ASSENTOS"])

media_dezembro_2019 = dezembro_2019.groupby(['ANO', 'MES']).agg(price_weighted_mean=("REAL", wm))
media_dezembro_2019

,,price_weighted_mean
ANO,MES,
2019,12,531.823898


In [68]:
(float(media_dezembro['price_weighted_mean']) - float(media_dezembro_2019['price_weighted_mean'])) / float(media_dezembro_2019['price_weighted_mean']) *100

22.374703943441464

In [11]:
dezembro.columns = ['ANO', 'MES', 'EMPRESA', 'ORIGEM', 'DESTINO', 'TARIFA', 'ASSENTOS']

In [12]:
dezembro.TARIFA = dezembro.TARIFA.str.replace(",", ".").astype(float)

In [28]:
len(dezembro)

360948

In [29]:
%%time
dezembro['REAL'] = dezembro.apply(lambda row: corretor(row["TARIFA"], f"{int(row.MES)}/{int(row.ANO)}", "07/2022")[0], axis=1) 

CPU times: user 8min 58s, sys: 5.61 s, total: 9min 4s
Wall time: 9min 16s


In [30]:
# Aqui com os dados base julho de 2022
wm = lambda x: np.average(x, weights=dezembro.loc[x.index, "ASSENTOS"])

dezembro.groupby(['ANO', 'MES']).agg(price_weighted_mean=("REAL", wm))

,,price_weighted_mean
ANO,MES,
2020,12,486.936127


In [27]:
# Ver se vai dar o mesmo resultado em dezembro.
wm = lambda x: np.average(x, weights=dezembro.loc[x.index, "ASSENTOS"])

dezembro.groupby(['ANO', 'MES']).agg(price_weighted_mean=("REAL", wm))

,,price_weighted_mean
ANO,MES,
2020,12,491.655068


In [23]:
# Ver se vai dar o mesmo resultado em dezembro.
wm = lambda x: np.average(x, weights=dezembro.loc[x.index, "ASSENTOS"])

dezembro.groupby(['ANO', 'MES']).agg(price_weighted_mean=("REAL", wm))

,,price_weighted_mean
ANO,MES,
2020,12,491.654549


In [51]:
%%time
dezembro['REAL'] = dezembro.apply(lambda row: corretor.ipca(row["TARIFA"], f"{int(row.MES)}/{int(row.ANO)}", "12/2022")[0], axis=1) 

CPU times: user 19min 31s, sys: 12.1 s, total: 19min 43s
Wall time: 20min 34s


In [53]:
dezembro

,ANO,MES,EMPRESA,ORIGEM,DESTINO,TARIFA,ASSENTOS,REAL
0,2022,12,ABJ,SBSV,SIRI,550.0,8,553.41
1,2022,12,ABJ,SBSV,SIRI,650.0,7,654.03
2,2022,12,ABJ,SBSV,SIRI,850.0,11,855.27
3,2022,12,ABJ,SBSV,SIRI,1050.0,7,1056.51
4,2022,12,ABJ,SBSV,SIRI,1250.0,3,1257.75
...,...,...,...,...,...,...,...,...
331370,2022,12,TAM,SWSI,SBVT,837.0,1,842.19
331371,2022,12,TAM,SWSI,SBVT,1395.0,1,1403.65
331372,2022,12,TAM,SWSI,SBVT,1813.9,1,1825.15
331373,2022,12,TAM,SWSI,SBVT,1866.0,1,1877.57


In [11]:
# %%time
# dezembro['REAL'] = dezembro.apply(lambda row: corretorOld.ipca(row["TARIFA"], f"{int(row.MES)}/{int(row.ANO)}", "12/2022")[0], axis=1) 

CPU times: user 22min 41s, sys: 15.4 s, total: 22min 56s
Wall time: 23min 44s


In [36]:

wm = lambda x: np.average(x, weights=dezembro.loc[x.index, "ASSENTOS"])

dezembro.groupby(['ANO', 'MES']).agg(price_weighted_mean=("REAL", wm))

,,price_weighted_mean
ANO,MES,
2022,12,654.853019


In [ ]:
wm = lambda x: np.average(x, weights=dezembro.loc[x.index, "ASSENTOS"])

dezembro.groupby(['ANO', 'MES']).agg(price_weighted_mean=("REAL", wm))

,,price_weighted_mean
ANO,MES,
2020,12,491.654549


In [ ]:
def get_average(values, weights):
    return float(np.average(values, weights=weights))    # function created to calculate wt. median


In [116]:
df["real"] = df[1:].apply(lambda row: corretor.ipca(row["Weighted_average"], f"{int(row.MES)}/{int(row.ANO)}", "12/2022")[0], axis=1) 

In [117]:
df[df["MES"]==12]

,ANO,MES,Total,Weighted_average,real
244,2022,12,1826543,650.817925,NaN
115,2021,12,2421438,490.797236,522.98
110,2020,12,1868805,416.657935,491.66
185,2019,12,2585896,436.533702,537.32
197,2018,12,2498232,444.033223,564.44
125,2017,12,2745175,430.039663,568.77
147,2016,12,2839457,363.795988,494.65
21,2015,12,3286404,358.082682,520.90
52,2014,12,3915556,333.243424,535.55
78,2013,12,3790080,351.360833,601.69
